# BEE 4750 Final Project: Solar Photovoltaic Feasibility Study on Cornell's Engineering Quadrangle

**Names**: Sitara Sastry, Maya Zoe Yu

**ID**: sls496, mzy3

> **Introduction**
>
> The following model will conduct a feasibility study of solar energy production on Cornell's Engineering Quadrangle. As Cornell’s Climate Action Plan seeks to have Cornell’s Ithaca campus carbon neutral by 2035 through utilizing 100% renewable energy, the proposed research will analyze if solar energy would help to achieve these goals. As such, the study will analyze how Cornell can invest in solar photovoltaics (PV) to increase this percentage – focusing mainly on the Engineering Quadrangle. In addition, the model will analyze the theoretical production of solar rooftop PV systems while optimzing cost. 


In [9]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

Pkg.add("JuMP")
Pkg.add("HiGHS")
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("GraphRecipes")
Pkg.add("Plots")
Pkg.add("Measures")
Pkg.add("MarkdownTables")

  Activating project at `c:\Users\Sitara Sastry\.julia\bee4750_solar`

   Resolving package versions...


  No Changes to `C:\Users\Sitara Sastry\.julia\bee4750_solar\Project.toml`
  No Changes to `C:\Users\Sitara Sastry\.julia\bee4750_solar\Manifest.toml`


   Resolving package versions...


  No Changes to `C:\Users\Sitara Sastry\.julia\bee4750_solar\Project.toml`
  No Changes to `C:\Users\Sitara Sastry\.julia\bee4750_solar\Manifest.toml`

...

   Resolving package versions...


  No Changes to `C:\Users\Sitara Sastry\.julia\bee4750_solar\Project.toml`
  No Changes to `C:\Users\Sitara Sastry\.julia\bee4750_solar\Manifest.toml`

   Resolving package versions...


  No Changes to `C:\Users\Sitara Sastry\.julia\bee4750_solar\Project.toml`
  No Changes to `C:\Users\Sitara Sastry\.julia\bee4750_solar\Manifest.toml`

   Resolving package versions...
  No Changes to `C:\Users\Sitara Sastry\.julia\bee4750_solar\Project.toml`
  No Changes to `C:\Users\Sitara Sastry\.julia\bee4750_solar\Manifest.toml`


### Theoretical Solar Output Model

#### Electricity Demand for Cornell's Engineering Quad
Cornell's Engineering Quadrangle encompasses the following buildings: Olin Hall, Carpenter Hall, Duffield Hall, Phillips Hall, Rhodes Hall, Upson Hall, Kimball Hall, Bard Hall, Snee Hall, Hollister Hall, and Ward Hall. The yearly electricity demand for these buildings may be found through the following: 

In [1]:
### Solar Geometry Function
function solar_geometry(N, L, H, β, γ)
    # Convert relevant inputs into radians 
    L = deg2rad(L)
    β = deg2rad(β)
    γ = deg2rad(γ)

    # Calculate all relevant solar geometry angles 
    δ = 23.45(sin(deg2rad(360(284+N))/365)) # Declination Angle
    ω = ((H-12)*15) # Hour Angle 
    δ = deg2rad(δ)
    ω = deg2rad(ω)
    α = asin(sin(δ)*sin(L)+cos(δ)*cos(L)*cos(ω)) # Solar Altitude Angle
    γ_s = asin((cos(δ)*sin(ω))/cos(α)) # Solar Azimuth Angle
    θ_i = acos(sin(α)*cos(β)+cos(α)*sin(β)*cos(γ-γ_s)) # Incidence Angle

    # Convert all relevant outputs back to degrees 
    δ = rad2deg(δ)
    ω = rad2deg(ω)
    α = rad2deg(α)
    γ_s = rad2deg(γ_s )
    θ_i = rad2deg(θ_i)

    # Return all outputs
    return δ, ω, α, γ_s, θ_i
end

### Solar Insolation - Direct, Diffuse, Reflected
function solar_insolation(N, L, β, K_t, ρ, H)
    ratio_diffuse_global = 1.39-4.03(K_t)+5.53(K_t)^2-3.11(K_t)^3 # Ratio of Diffuse to Global Insolation
    # Calculate sunset hour angle based on N
    if N >= 79 && N <= 265
        ω_s = min(acos(-tan(L)*tan(δ)), acos(-tan(L-β)*tan(δ)))
    else 
        ω_s = acos(-tan(L)*tan(δ))
    end
    # Ratio of Beam at Angle to Beam onto Horizontal Surface
    R_{b,β}=[cos(L-β)cos(δ)sin(ω_s)+ω_ssin(L-β)sin(δ)]/[cos(L)cos(δ)sin(ω_s)+ω_ssin(L)sin(δ)] 
    # Calculate direct, diffuse, and reflected insolation
    Direct = (1-ratio_diffuse_global)R_{b,β}
    Diffuse = (ratio_diffuse_global)[1+cos(β)]/2
    Reflected = ρ(1-cos(β))/2
    # Calcuate ratio of global insolation at angle to horizontal
    R = Direct+Diffuse+Reflected
    # Calculate total insolation onto angled surface at angle β [Energy/Area]
    H_t = H(R)
end


solar_insolation (generic function with 1 method)

In [34]:
using CSV
using DataFrames

### Annual Energy Demand
bard = DataFrame(CSV.File("BardElectricData.csv"));
bardDemand = sum(bard.Data[1:end]);
carpenter = DataFrame(CSV.File("CarpenterElectricData.csv"));
carpenterDemand = sum(carpenter.Data[1:end]);
duffield = DataFrame(CSV.File("DuffieldElectricData.csv"));
duffieldDemand = sum(duffield.Data[1:end]);
hollister = DataFrame(CSV.File("HollisterElectricData.csv"));
hollisterDemand = sum(hollister.Data[1:end]);
kimball = DataFrame(CSV.File("KimballElectricData.csv"));
kimballDemand = sum(kimball.Data[1:end]);
olin = DataFrame(CSV.File("OlinElectricData.csv"));
olinDemand = sum(olin.Data[1:end]);
phillips = DataFrame(CSV.File("PhillipsElectricData.csv"));
phillipsDemand = sum(phillips.Data[1:end]);
snee = DataFrame(CSV.File("SneeElectricData.csv"));
sneeDemand = sum(snee.Data[1:end]);
rhodes = DataFrame(CSV.File("RhodeElectricData.csv"));
rhodesDemand = sum(rhodes.Data[1:end]);
upson = DataFrame(CSV.File("UpsonElectricData.csv"));
upsonDemand = sum(upson.Data[1:end]);
ward = DataFrame(CSV.File("WardElectricData.csv"));
wardDemand = sum(ward.Data[1:end]);

### Rooftop Area (in meters)
bardArea = 1060.34;
carpenterArea = 1670.21;
duffieldArea = 1971.78;
hollisterArea = 1965.75;
kimballArea = 956.47;
olinArea = 1529.58;
phillipsArea = 1665.13;
sneeArea = 1680.23;
rhodesArea = 3824.18;
upsonArea = 2176.19;
wardArea = 1033.94;

#### Future Steps 
- Assume beta, L, K_t, rho, H, gamma 
- Create data frame for N (1:365)
